# Evaluation of the Classifier

This is the evaluation script of the classifier that outputs a vector of size 5.

### library and package importation

In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

from graphs import draw
import graphs

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras.models import load_model
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

from IPython.display import clear_output

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

sample_rate=44100
hop_length = 441  # frame size= 2*hop
segment_length=int(sample_rate*0.2)  #0.2
segment_pad=int(sample_rate*0.02)     #0.02
overlappiong=int(sample_rate*0.1)   #0.1

NumofFeaturetoUse = 272 # this will re-assigned for different classifiers
frame_number = 48

sys.path.insert(1, '..//components//')
import load_feat_directories

# input new indices file here
# indices_filename = 'D://indices_filename.npy'
# indices=np.load(indices_filename)

Using TensorFlow backend.


In [2]:
def avail_modules(directory, prefix):
    module_names = []
    for item in os.listdir(directory):
        if prefix in item:
            module_names.append(directory + item)
            i = module_names.index(directory + item)
            print(str(i) + 'th module')
            print(directory + item)
    return module_names

In [3]:
def comprise_vector(path, dist, reverb, noise):
    vec_to_return = []
    for fname in os.listdir(path):
        components = fname.split('_')
        '''
        if dist == 0 and 'deamp' in components: continue
        if reverb == 0 and 'WetDry' in components: continue
        if noise == 0 and 'noise' in components: continue
        '''
        current_vec = np.load(path + fname)
        vec_to_return.append(current_vec)
        
    vec_to_return = np.array(vec_to_return)
    return vec_to_return

In [4]:
def comprise_label(feature_vector, label):
    label_vec_to_ret = []
    length = len(list(feature_vector))
    for index in range(0, length):
        current_label = [label]
        label_vec_to_ret.append(current_label)
    label_vec_to_ret = np.array(label_vec_to_ret)

    return label_vec_to_ret

def float_compatible(input_np):

    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)

    return input_np

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [5]:
# allnoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
# homenoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
all_noised_npy = load_feat_directories.allnoised_npy
all_noised_npy_test = load_feat_directories.allnoised_npy_test
home_noised_npy = load_feat_directories.homenoised_npy
home_noised_npy_test = load_feat_directories.homenoised_npy_test

for index in range(0, 5):
    if not os.path.exists(all_noised_npy_test[index]):
        print(all_noised_npy_test[index] + ' does not exist. Breaking the loop... ')
    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy_test[index] + 'does not exist. Breaking the loop... ')

In [18]:
emotions = [0, 1, 2, 3, 4]
home_or_all = 'home'
dist = 0
reverb = 0
noise = 0

for index in emotions:
  
    if index == 0:
        
        path = home_noised_npy_test[index]
        val_h_feat = comprise_vector(path, dist, reverb, noise)
        val_h_label = comprise_label(val_h_feat, index)
        
        path = home_noised_npy[index]
        h_feat = comprise_vector(path, dist, reverb, noise)
        h_label = comprise_label(h_feat, index)
        
    elif index == 1:
        path = home_noised_npy_test[index]
        val_a_feat = comprise_vector(path, dist, reverb, noise)
        val_a_label = comprise_label(val_a_feat, index)
        
        path = home_noised_npy[index]
        a_feat = comprise_vector(path, dist, reverb, noise)
        a_label = comprise_label(a_feat, index)
    elif index == 2:
        path = home_noised_npy_test[index]
        val_n_feat = comprise_vector(path, dist, reverb, noise)
        val_n_label = comprise_label(val_n_feat, index)
        
        path = home_noised_npy[index]
        n_feat = comprise_vector(path, dist, reverb, noise)
        n_label = comprise_label(n_feat, index)
    elif index == 3:
        path = home_noised_npy_test[index]
        val_s_feat = comprise_vector(path, dist, reverb, noise)
        val_s_label = comprise_label(val_s_feat, index)
        
        path = home_noised_npy[index]
        s_feat = comprise_vector(path, dist, reverb, noise)
        s_label = comprise_label(s_feat, index)
    else:
        path = home_noised_npy_test[index]
        val_o_feat = comprise_vector(path, dist, reverb, noise)
        val_o_label = comprise_label(val_o_feat, index)
        
        path = home_noised_npy[index]
        o_feat = comprise_vector(path, dist, reverb, noise)
        o_label = comprise_label(o_feat, index)


# Count files in different scenarios

In [38]:
def eval_all(emotionFolders):
    total = 0
    h = 0
    a = 0
    nn = 0
    s = 0
    o = 0
    
    for emotionFolder in emotionFolders:
        for emotionfile in os.listdir(emotionFolder):
            if 'Happy' in emotionfile: h += 1
            elif 'Angry' in emotionfile: a += 1
            elif 'Neutral' in emotionfile: nn += 1
            elif 'Sad' in emotionfile: s += 1
            else: o += 1

            total += 1

    return [h, a, nn, s, o, total]

In [39]:
def eval_original(emotionFolders):
    total = 0
    h = 0
    a = 0
    nn = 0
    s = 0
    o = 0
    
    for emotionFolder in emotionFolders:
        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile and 'Diffusion_' in emotionfile and 'DecayFactor_' in emotionfile
            noise = 'noise_' in emotionfile
        
            r_n = not dist and reverb and noise 
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise 
            
            if original:           
                if 'Happy' in emotionfile: h += 1
                elif 'Angry' in emotionfile: a += 1
                elif 'Neutral' in emotionfile: nn += 1
                elif 'Sad' in emotionfile: s += 1
                else: o += 1
                
                total += 1
    
    return [h, a, nn, s, o, total]

In [54]:
def eval_deamplified_noised(emotionFolders):
    total = 0
    h = 0
    a = 0
    nn = 0
    s = 0
    o = 0
    
    for emotionFolder in emotionFolders:
        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile and 'Diffusion_' in emotionfile and 'DecayFactor_' in emotionfile
            noise = 'noise_' in emotionfile
        
            r_n = not dist and reverb and noise 
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise # does not exist
            
            if d_n:
                if 'Happy' in emotionfile: h += 1
                elif 'Angry' in emotionfile: a += 1
                elif 'Neutral' in emotionfile: nn += 1
                elif 'Sad' in emotionfile: s += 1
                else: o += 1
                    
                total += 1
    return [h, a, nn, s, o, total]

In [55]:
def eval_reverbed(emotionFolders):
    total = 0
    h = 0
    a = 0
    nn = 0
    s = 0
    o = 0
    
    for emotionFolder in emotionFolders:

        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile and 'Diffusion_' in emotionfile and 'DecayFactor_' in emotionfile
            noise = 'noise_' in emotionfile
        
            r_n = not dist and reverb and noise 
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise # does not exist
            
            if r:
                if 'Happy' in emotionfile: h += 1
                elif 'Angry' in emotionfile: a += 1
                elif 'Neutral' in emotionfile: nn += 1
                elif 'Sad' in emotionfile: s += 1
                else: o += 1
                    
                total += 1         
    return [h, a, nn, s, o, total]

In [56]:
def eval_deamplified_noised_reverbed(emotionFolders):
    total = 0
    h = 0
    a = 0
    nn = 0
    s = 0
    o = 0
    for emotionFolder in emotionFolders:
        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile and 'Diffusion_' in emotionfile and 'DecayFactor_' in emotionfile
            noise = 'noise_' in emotionfile
        
            r_n = not dist and reverb and noise 
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise # does not exist
            
            if d_r_n:
                if 'Happy' in emotionfile: h += 1
                elif 'Angry' in emotionfile: a += 1
                elif 'Neutral' in emotionfile: nn += 1
                elif 'Sad' in emotionfile: s += 1
                else: o += 1
                    
                total += 1
    return [h, a, nn, s, o, total]

In [57]:
# Load testing npy files
featureSet_val = float_compatible(np.vstack((val_h_feat, val_a_feat, val_n_feat, val_s_feat)))
Label_val = np.vstack((val_h_label, val_a_label, val_n_label, val_s_label))

Label_val = to_categorical(Label_val)
print('testing data: ' + str(featureSet_val.shape))
print('testing label: ' + str(Label_val.shape))

testing data: (7551, 48, 272)
testing label: (7551, 4)


In [58]:
# Load testing npy files
featureSet = float_compatible(np.vstack((h_feat, a_feat, n_feat, s_feat)))
Label = np.vstack((h_label, a_label, n_label, s_label))

Label = to_categorical(Label)
print('evaluation data: ' + str(featureSet.shape))
print('evaluation label: ' + str(Label.shape))

evaluation data: (34155, 48, 272)
evaluation label: (34155, 4)


## Specify the directories that contain the files of emotion

In [71]:
emotionFolders_val = [home_noised_npy_test[0], home_noised_npy_test[1], \
                  home_noised_npy_test[2], home_noised_npy_test[3], home_noised_npy_test[4]]

In [79]:
emotionFolders = [home_noised_npy[0], home_noised_npy[1], \
                  home_noised_npy[2], home_noised_npy[3], home_noised_npy[4]]

In [80]:
val_s1 = eval_original(emotionFolders_val)
val_s2 = eval_deamplified_noised(emotionFolders_val)
val_s3 = eval_reverbed(emotionFolders_val)
val_s4 = eval_deamplified_noised_reverbed(emotionFolders_val)
val_s5 = eval_all(emotionFolders_val)

In [81]:
train_s1 = eval_original(emotionFolders)
train_s2 = eval_deamplified_noised(emotionFolders)
train_s3 = eval_reverbed(emotionFolders)
train_s4 = eval_deamplified_noised_reverbed(emotionFolders)
train_s5 = eval_all(emotionFolders)

In [82]:
def display(val, train):
    h = val[0] + train[0]
    a = val[1] + train[1]
    n = val[2] + train[2]
    s = val[3] + train[3]
    o = val[4] + train[4]
    t = val[5] + train[5]
    
    print('There are ' + str(h) + ' happy samples, ' +  str(a) + ' angry samples, ' + \
          str(n) + ' neutral samples, ' + str(s) + ' sad samples, ' + 
         str(o) + ' samples of confounding speech. In total, there are ' + str(t) + ' samples.' )

In [83]:
display(val_s1, train_s1)

There are 1792 happy samples, 1793 angry samples, 1573 neutral samples, 1793 sad samples, 1837 samples of confounding speech. In total, there are 8788 samples.


In [84]:
display(val_s2, train_s2)

There are 3584 happy samples, 3586 angry samples, 3146 neutral samples, 3586 sad samples, 3674 samples of confounding speech. In total, there are 17576 samples.


In [85]:
display(val_s3, train_s3)

There are 1792 happy samples, 1793 angry samples, 1573 neutral samples, 1793 sad samples, 1542 samples of confounding speech. In total, there are 8493 samples.


In [86]:
display(val_s4, train_s4)

There are 3584 happy samples, 3586 angry samples, 3146 neutral samples, 3586 sad samples, 0 samples of confounding speech. In total, there are 13902 samples.


In [87]:
display(val_s5, train_s5)

There are 10752 happy samples, 10758 angry samples, 9438 neutral samples, 10758 sad samples, 7053 samples of confounding speech. In total, there are 48759 samples.
